# Chainerで線形回帰してみる

線形回帰についてはtensorflowを参考。ここではChainer独自のところを説明していく

In [1]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

ChainerではVariableを使う。Chainerでは、入力の次元の配列を作らないといけないことに注意する。一次元の入力であれば、`[[data1],[data2],...]`というVariableを使う。(cf. Tensorflowでは`[data1, data2, ...]`であった。)

In [2]:
x_data = np.array([2], dtype=np.float32)
x = Variable(x_data)
print(x)
print("x.data ->", x.data)
print("x.grad ->", x.grad) # この段階でgradはNone
y = x ** 2
print("y.data->", y.data)

<var@110ce3c50>
x.data -> [ 2.]
x.grad -> None
y.data-> [ 4.]


アウトプットに対して、`backward()`とすると、その関数を作っているVariableでのgradの値が入力される。この場合はxでの偏微分の値。

In [3]:
y.backward()
print(x.grad)
y.zerograd() # 偏微分の値は蓄積されていくので、zerogradで初期化

[ 4.]


Chainerが面白いのは、F.LinerやL.Linerが
$$ \vec{y} = W\vec{x} +\vec{b}$$
の形を持った関数で、$W$や$\vec{b}$はランダムに初期化してくれている。

`F.Liner(1,1)`だと、1次元→1次元で、あとでMNISTなどで使う784次元→10次元であれば`F.Liner(784,10)`とすれば、&W&は784x10の行列で、$\vec{b}\in\mathbb{R}^{10}$となるように作ってくれる

In [4]:
f = F.Linear(1,1) # 1次元→1次元の y = Wx + b
print(f.W.data)
print(f.b.data)

[[-0.07353922]]
[ 0.]


100個のデータセットを作って、コスト関数を平均二乗誤差関数で定義してみる。

In [5]:
x_data = np.random.rand(100,1).astype("float32")
y_data = x_data * 0.1 + 0.3
x = Variable(x_data)
y = Variable(y_data)
loss = F.mean_squared_error(f(x), y) # 平均二乗誤差
loss.data

array(0.15548615157604218, dtype=float32)

バッチで一気に計算すると器用に偏微分の値は蓄積されていくみたいなので、偏微分を計算する前に初期化しておくための`zerograds`が準備されているので呼ぶ。

In [6]:
f.zerograds() # 微分値の初期化
loss.backward() # それぞれの偏微分を計算する
print(f.W.grad) # fのWでの変微分の値が求まる
print(f.b.grad) # fのbでの変微分の値が求まる

[[-0.44013578]]
[-0.78197145]


コスト関数の最小化の手法はoptimizersに準備されている。最急降下法がなさそうなので、確率的最急降下法を使う。`setup`で変数の更新対象の関数をセットする。そして、`update`でパラメーターを更新してくれる。

In [7]:
optimizer = optimizers.SGD(0.5) # 確率的最急降下法
optimizer.setup(f)
optimizer.update()
print(f.W.data)
print(f.b.data)

[[ 0.14652866]]
[ 0.39098573]


上記を合わせてイテレーションして学習を進めてみる

In [8]:
for step in range(201):
    loss = F.mean_squared_error(f(x), y)  # 再度コスト関数を計算する
    f.zerograds() # gradは保存されていくので初期化する
    loss.backward() # 微分を計算
    optimizer.update() # 確率的最急降下法で fをupdateする
    if step % 20 == 0:
        print(step,f.W.data, f.b.data)

0 [[ 0.08199314]] [ 0.27560526]
20 [[ 0.09918599]] [ 0.30045748]
40 [[ 0.09979708]] [ 0.30011407]
60 [[ 0.09994941]] [ 0.30002844]
80 [[ 0.09998742]] [ 0.30000708]
100 [[ 0.09999687]] [ 0.30000177]
120 [[ 0.09999921]] [ 0.30000046]
140 [[ 0.0999998]] [ 0.30000013]
160 [[ 0.0999999]] [ 0.30000007]
180 [[ 0.0999999]] [ 0.30000007]
200 [[ 0.0999999]] [ 0.30000007]


綺麗に、学習されている。

次に、Chainというクラスを使うことにより、もう少し便利に、上記が出来ることを見ていく。深層学習などの複雑なニューラルネットワークを生成するには、こちらを使うほうが便利そうだ。やっていることは上記と同じなので、説明は省く。

In [9]:
class LinearRegression(Chain):
    def __init__(self):
        super(LinearRegression, self).__init__(
            l1 = L.Linear(1,1)
        )
        
    def __call__(self, x, y):
        return F.mean_squared_error(self.l1(x), y)

In [10]:
model = LinearRegression()
optimizer = optimizers.SGD(0.5)
optimizer.setup(model)

In [11]:
import numpy as np
x_data = np.random.rand(100,1).astype("float32")
y_data = x_data * 0.1 + 0.3
print("x_data -> ", x_data[:5])
print("y_data -> ", y_data[:5])

x_data ->  [[ 0.21073872]
 [ 0.36733896]
 [ 0.54460317]
 [ 0.90479493]
 [ 0.77130264]]
y_data ->  [[ 0.32107389]
 [ 0.33673391]
 [ 0.35446033]
 [ 0.39047951]
 [ 0.37713027]]


In [12]:
for step in range(201):
    train_x = Variable(x_data)
    train_y = Variable(y_data)
    optimizer.update(model, train_x, train_y)
    if step % 20 == 0:
        print(step, model.l1.W.data, model.l1.b.data) 
    
#     model.zerograds()
#     loss = model(train_x, train_y)
#     loss.backword()
#     optimizer.update()

0 [[ 1.33077836]] [-0.72787428]
20 [[ 0.44749212]] [ 0.08820534]
40 [[ 0.18916766]] [ 0.24565278]
60 [[ 0.12288073]] [ 0.28605431]
80 [[ 0.10587127]] [ 0.2964215]
100 [[ 0.10150659]] [ 0.29908174]
120 [[ 0.10038661]] [ 0.29976436]
140 [[ 0.10009921]] [ 0.29993954]
160 [[ 0.10002545]] [ 0.29998451]
180 [[ 0.10000654]] [ 0.29999602]
200 [[ 0.10000168]] [ 0.299999]


綺麗に学習できている。